## 导入需要的包

In [1]:
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

In [2]:
## 加载数据

In [3]:
train = pd.read_csv('/home/mw/input/homestayprice/训练集.csv')
test = pd.read_csv('/home/mw/input/homestayprice/测试集.csv')

In [4]:
# 查看列名
train.columns

Index(['数据ID', '容纳人数', '便利设施', '洗手间数量', '床的数量', '床的类型', '卧室数量', '取消条款', '所在城市',
       '清洁费', '首次评论日期', '房主是否有个人资料图片', '房主身份是否验证', '房主回复率', '何时成为房主',
       '是否支持随即预订', '最近评论日期', '维度', '经度', '民宿周边', '评论个数', '房产类型', '民宿评分', '房型',
       '邮编', '价格'],
      dtype='object')

In [5]:
#打印数据大小
print(train.shape,test.shape)

(59288, 26) (14823, 25)


In [6]:
# 查看数据集缺失值
train.shape[0]-train.count()

数据ID               0
容纳人数               0
便利设施               0
洗手间数量            160
床的数量             104
床的类型               0
卧室数量              78
取消条款               0
所在城市               0
清洁费                0
首次评论日期         12702
房主是否有个人资料图片      160
房主身份是否验证         160
房主回复率          14641
何时成为房主           160
是否支持随即预订           0
最近评论日期         12672
维度                 0
经度                 0
民宿周边            5509
评论个数               0
房产类型               0
民宿评分           13395
房型                 0
邮编               761
价格                 0
dtype: int64

In [7]:
# 查看数据类型
train.dtypes

数据ID            object
容纳人数             int64
便利设施            object
洗手间数量          float64
床的数量           float64
床的类型             int64
卧室数量           float64
取消条款             int64
所在城市             int64
清洁费              int64
首次评论日期          object
房主是否有个人资料图片     object
房主身份是否验证        object
房主回复率           object
何时成为房主          object
是否支持随即预订         int64
最近评论日期          object
维度             float64
经度             float64
民宿周边            object
评论个数             int64
房产类型             int64
民宿评分           float64
房型               int64
邮编              object
价格             float64
dtype: object

In [8]:
# 价格密度分布
train['价格'].plot.kde()

<Figure size 432x288 with 1 Axes>

## 数据预处理 

### 类别变量数据预处理

In [9]:
train.select_dtypes(include=['object']).columns

Index(['数据ID', '便利设施', '首次评论日期', '房主是否有个人资料图片', '房主身份是否验证', '房主回复率', '何时成为房主',
       '最近评论日期', '民宿周边', '邮编'],
      dtype='object')

In [10]:
train['便利设施数量']=train['便利设施'].apply(lambda x:len(x.lstrip('{').rstrip('}').split(',')))
test['便利设施数量']=test['便利设施'].apply(lambda x:len(x.lstrip('{').rstrip('}').split(',')))

train['便利设施数量'].head()

0    17
1    18
2    11
3     1
4    13
Name: 便利设施数量, dtype: int64

In [11]:
no_features = ['数据ID', '价格','便利设施']

In [12]:
# 其他类别变量，暂时先直接简单的类别编码
data = pd.concat([train, test], axis=0)
for col in train.select_dtypes(include=['object']).columns:
    if col not in no_features:
        lb = LabelEncoder()
        lb.fit(data[col].astype(str))
        train[col] = lb.transform(train[col].astype(str))
        test[col] = lb.transform(test[col].astype(str))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [13]:
train.head(2)

,数据ID,容纳人数,便利设施,洗手间数量,床的数量,床的类型,卧室数量,取消条款,所在城市,清洁费,...,维度,经度,民宿周边,评论个数,房产类型,民宿评分,房型,邮编,价格,便利设施数量
0,train_0,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.5,3.0,4,2.0,0,3,0,...,34.109039,-118.273390,323,12,17,97.0,0,454,64.918531,17
1,train_1,2,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",1.0,1.0,4,1.0,2,4,1,...,40.812897,-73.919163,371,6,0,87.0,0,150,54.918531,18


In [14]:
# 输入特征列
features = [col for col in train.columns if col not in no_features]
features

['容纳人数',
 '洗手间数量',
 '床的数量',
 '床的类型',
 '卧室数量',
 '取消条款',
 '所在城市',
 '清洁费',
 '首次评论日期',
 '房主是否有个人资料图片',
 '房主身份是否验证',
 '房主回复率',
 '何时成为房主',
 '是否支持随即预订',
 '最近评论日期',
 '维度',
 '经度',
 '民宿周边',
 '评论个数',
 '房产类型',
 '民宿评分',
 '房型',
 '邮编',
 '便利设施数量']

In [15]:
X = train[features] # 训练集输入
y = train['价格'] # 训练集标签
X_test = test[features] # 测试集输入

## 模型训练

- 定义一个LightGBM回归模型
- 进行5折交叉验证训练


In [16]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True,random_state=1314)

In [17]:
# 祖传参数
params = {
    'learning_rate': 0.1,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_leaves': 1000,
    'verbose': -1,
    'max_depth': -1,
    'seed': 2019,
    # 'n_jobs': -1,
    # 'device': 'gpu',
    # 'gpu_device_id': 0,
}



In [27]:
oof = np.zeros(len(X))
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
    X_train, X_valid = X[features].iloc[train_index], X[features].iloc[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    model = lgb.LGBMRegressor(**params, n_estimators=50000, n_jobs=-1)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric='rmse',
              verbose=50, early_stopping_rounds=200)
    y_pred_valid = model.predict(X_valid)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
    oof[valid_index] = y_pred_valid.reshape(-1, )
    prediction += y_pred
prediction /= n_fold

In [20]:
# 验证集评估

In [21]:
from sklearn.metrics import mean_squared_error
score=mean_squared_error(oof,train['价格'].values,squared=False)
score

## 提交结果

In [24]:
test['价格'] = prediction
test[['数据ID', '价格']].to_csv('result/sub.csv'.format(score), index=None)

In [25]:
test[['数据ID', '价格']].head()

## TODO
- 可以做更细粒度的特征
- 尝试不同的模型
- 模型融合
- ...